In [7]:
import torch
from datasets import load_dataset
from transformers import AutoTokenizer

In [4]:
data_files={"train":"swerick_data_random_train.pkl","test":"swerick_data_random_test.pkl","valid":"swerick_data_random_valid.pkl"}
swerick_dataset = load_dataset("pandas",data_files=data_files)
swerick_dataset

Generating train split: 0 examples [00:00, ? examples/s]

Generating test split: 0 examples [00:00, ? examples/s]

Generating valid split: 0 examples [00:00, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['protocole', 'texte'],
        num_rows: 12319
    })
    test: Dataset({
        features: ['protocole', 'texte'],
        num_rows: 2683
    })
    valid: Dataset({
        features: ['protocole', 'texte'],
        num_rows: 2683
    })
})

In [19]:
def get_training_corpus():
    for i in range(0, len(swerick_dataset["train"]),1000):
        yield swerick_dataset["train"][i:i+1000]["texte"]

In [6]:
from tokenizers import (
    decoders,
    models,
    normalizers,
    pre_tokenizers,
    processors,
    trainers,
    Tokenizer,
)

tokenizer = Tokenizer(models.WordPiece(unk_token="[UNK]"))

In [7]:
tokenizer.normalizer = normalizers.BertNormalizer(lowercase=False,strip_accents=False)


In [8]:
tokenizer.pre_tokenizer = pre_tokenizers.Whitespace()

In [10]:
tokenizer.pre_tokenizer.pre_tokenize_str(swerick_dataset["train"][100]["texte"])

[('Den', (0, 3)),
 ('3', (4, 5)),
 ('Maj', (6, 9)),
 ('.', (9, 10)),
 ('289', (11, 14)),
 ('Fredagen', (15, 23)),
 ('den', (24, 27)),
 ('3', (28, 29)),
 ('Maj', (30, 33)),
 ('1867', (34, 38)),
 ('.', (38, 39)),
 ('Kammaren', (40, 48)),
 ('sammanträdde', (49, 61)),
 ('kl', (62, 64)),
 ('.', (64, 65)),
 ('2', (66, 67)),
 ('e', (68, 69)),
 ('.', (69, 70)),
 ('m', (71, 72)),
 ('.', (72, 73)),
 ('Justerades', (74, 84)),
 ('protokollet', (85, 96)),
 ('för', (97, 100)),
 ('den', (101, 104)),
 ('26', (105, 107)),
 ('sistlidne', (108, 117)),
 ('Mars', (118, 122)),
 ('f', (123, 124)),
 ('.', (124, 125)),
 ('m', (126, 127)),
 ('.', (127, 128)),
 ('Upplästes', (129, 138)),
 ('och', (139, 142)),
 ('godkändes', (143, 152)),
 ('Stats', (153, 158)),
 ('-', (158, 159)),
 ('Utskottets', (159, 169)),
 ('förslag', (170, 177)),
 ('till', (178, 182)),
 ('Riksdagens', (183, 193)),
 ('underdåniga', (194, 205)),
 ('skrifvelser', (206, 217)),
 (':', (217, 218)),
 ('N', (219, 220)),
 (':', (220, 221)),
 ('o', (2

In [11]:
special_tokens = ["[UNK]", "[PAD]", "[CLS]", "[SEP]", "[MASK]"]
trainer = trainers.WordPieceTrainer(vocab_size=50325, special_tokens=special_tokens)

In [20]:
tokenizer.train_from_iterator(get_training_corpus(), trainer=trainer)

In [21]:
encoding = tokenizer.encode(swerick_dataset["train"][100]["texte"])
print(encoding.tokens)

['Den', '3', 'Maj', '.', '289', 'Fredagen', 'den', '3', 'Maj', '1867', '.', 'Kammaren', 'sammanträdde', 'kl', '.', '2', 'e', '.', 'm', '.', 'Justerades', 'protokollet', 'för', 'den', '26', 'sistlidne', 'Mars', 'f', '.', 'm', '.', 'Upplästes', 'och', 'godkändes', 'Stats', '-', 'Utskottets', 'förslag', 'till', 'Riksdagens', 'underdåniga', 'skrifvelser', ':', 'N', ':', 'o', '40', ',', 'angående', 'eftergift', 'af', 'Kronans', 'rätt', 'till', 'ett', 'danaarf', ';', 'N', ':', 'o', '41', ',', 'i', 'anledning', 'af', 'Kongl', '.', 'Maj', ':', 'ts', 'nådiga', 'Proposition', 'om', 'den', 'under', 'N', ':', 'o', '2', 'Skepp', '##sbron', 'i', 'hufvudstaden', 'belägna', ',', 'Kronan', 'tillhöriga', 'f', '.', 'd', '.', 'Ridder', '##stolpe', '##ska', 'egendomens', 'användande', 'till', 'byggnad', 'för', 'Telegraf', '##verket', ';', 'och', 'N', ':', 'o', '42', ',', 'i', 'anledning', 'af', 'år', '1866', 'verkställd', 'revision', 'af', 'Statsverkets', 'med', 'flera', 'allmänna', 'fonders', 'förvaltning

In [22]:
cls_token_id = tokenizer.token_to_id("[CLS]")
sep_token_id = tokenizer.token_to_id("[SEP]")
print(cls_token_id, sep_token_id)

2 3


In [23]:
tokenizer.post_processor = processors.TemplateProcessing(
    single=f"[CLS]:0 $A:0 [SEP]:0",
    pair=f"[CLS]:0 $A:0 [SEP]:0 $B:1 [SEP]:1",
    special_tokens=[("[CLS]", cls_token_id), ("[SEP]", sep_token_id)],
)

In [24]:
tokenizer.decoder = decoders.WordPiece(prefix="##")

In [25]:
tokenizer.decode(encoding.ids)

'Den 3 Maj. 289 Fredagen den 3 Maj 1867. Kammaren sammanträdde kl. 2 e. m. Justerades protokollet för den 26 sistlidne Mars f. m. Upplästes och godkändes Stats - Utskottets förslag till Riksdagens underdåniga skrifvelser : N : o 40, angående eftergift af Kronans rätt till ett danaarf ; N : o 41, i anledning af Kongl. Maj : ts nådiga Proposition om den under N : o 2 Skeppsbron i hufvudstaden belägna, Kronan tillhöriga f. d. Ridderstolpeska egendomens användande till byggnad för Telegrafverket ; och N : o 42, i anledning af år 1866 verkställd revision af Statsverkets med flera allmänna fonders förvaltning under år 1864, Föredrogs ånyo Andra Kammarens den I och 2 dennes bordlagda protokollsutdrag N : o 304, med delgifvande af Kammarens beslut öfver dess Tillfälliga Utskotts Utlåtande N : o 39, i anledning af återremiss utaf samma Utskotts Utlåtande N : o 19 öfver Herr Sven Hanssons motion N : o 257, angående ändring af 6 $ 3 mom. i 1842 årg folkskolestadga. Herr Björck : I öfverensstämmel

In [26]:
tokenizer.save("tokenizer_swerick.json")

In [5]:
from transformers import PreTrainedTokenizerFast

wrapped_tokenizer = PreTrainedTokenizerFast(
    #tokenizer_object=tokenizer,
    tokenizer_file="tokenizer_swerick.json", # You can load from the tokenizer file, alternatively
    unk_token="[UNK]",
    pad_token="[PAD]",
    cls_token="[CLS]",
    sep_token="[SEP]",
    mask_token="[MASK]",
)

In [4]:
from transformers import BertTokenizerFast

wrapped_tokenizer = BertTokenizerFast(tokenizer_object=tokenizer)

NameError: name 'tokenizer' is not defined

In [9]:
#difference de tokenizer 
base_tokenizer = AutoTokenizer.from_pretrained("KBLab/bert-base-swedish-cased")

In [10]:
replace =lambda x :x.replace('##',"")
swerick_voc=list(map(replace,wrapped_tokenizer.vocab.keys()))
base_voc=list(map(replace,base_tokenizer.vocab.keys()))

In [14]:
def get_vocab_sim(first_vocab_keys,second_vocab_keys):
    f_set=set(first_vocab_keys)
    s_set=set(second_vocab_keys)
    intersection = f_set.intersection(s_set)
    union = f_set.union(s_set)

    similarity_jaccard= len(intersection)/len(union)
    vocab_f = f_set-s_set
    return intersection, len(intersection)/len(f_set), len(intersection)/len(s_set),similarity_jaccard,vocab_f

In [15]:
inter,f,s,jaccard,vocab_f=get_vocab_sim(swerick_voc,base_voc)
print(inter)
print(f,s)
print("similarity of Jaccard",jaccard)
print("New Vocab added in tokenizer of swerick", vocab_f)

{'personlighet', 'broschyren', 'sorgliga', 'inlägg', 'hastigt', 'professur', 'ammar', 'del', 'kyrkan', 'uppehöll', 'energi', 'istisk', 'hjälpte', 'Ur', 'devalver', 'utbytt', 'hämtning', 'utanförskap', 'medborgerliga', 'omhändertagande', '1966', 'progress', 'driven', 'ändra', 'â', 'alterna', 'havs', 'skostnad', 'hetens', 'välja', 'miljövän', 'IO', 'ändska', 'UL', 'Blix', 'särklass', 'landshövding', 'turist', 'ighetens', 'orkester', 'bekräftelse', 'budet', 'branschen', 'tom', 'konstaterats', 'lige', 'glädjen', 'ces', 'väls', 'parlamentariska', 'Närings', '606', 'onorm', 'Ullsten', 'fisk', 'Finans', 'långvarigt', 'vis', 'skont', 'terna', 'Mattson', 'Klart', 'säll', 'Il', 'gem', '/', 'kass', 'aktierna', 'elektriska', 'sakna', 'muslim', 'kommuna', 'bosättning', 'härskar', 'dyr', 'dödande', 'väst', 'bestått', 'sbolagets', 'koncentrationen', 'Wh', 'ärn', 'intagning', 'yndigheterna', 'läst', 'TO', 'kalkyl', 'strök', 'utforma', 'chs', 'pri', 'tråk', 'Kristen', 'rationellt', 'hästa', 'Gud', 'för

In [ ]:
same_words=0
wrong_words=0
wrong_list=[]
for word in list(swerick_voc):
    base_token = base_tokenizer.tokenize(word)